<a href="https://colab.research.google.com/github/Yuvaranii/Library_Mgmt_Sys/blob/main/LMS_Try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install dnspython
!pip install pymongo[srv] 
import pymongo
import re
import pandas as pd
import datetime as dt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
Client = pymongo.MongoClient("mongodb+srv://@@@:@@@@cluster0.os1btpz.mongodb.net/?retryWrites=true&w=majority")
db = Client.LMS
Reg = db.Register
Inv = db.Inventory
Iss = db.Issued

In [ ]:
def Phone_Num():
  I = input ("Enter the Phone Number" "\n")
  R = "[6-9][0-9]{9}$"
  if re.match (R,I):
    if Reg.find_one({"_id" : {"$eq" : I}}):
      print ("Phone number already exists")
      Home()
    else:
      print ("Phone number is Valid")
      return (I)
  else: 
    print ("Enter the Valid Phone Number \nEnter the 10 digit Mobile Number, it should starts with (6-9)")

def PW():
  I = input ("Enter the Password" "\n")
  P="^(?=.*[a-z])(?=.*[A-Z])(?=.*[0-9])(?=.*[^A-Za-z0-9])(?=.{6,16})"
  if re.match (P,I):
    print ("Password Validated")
    return (I)
  else:
    print("Invalid password, \nPlease enter a valid password with 1 number, \n1 lowercase, 1 upper case, \n1 special character within 6 to 15 characters")

def Student_Name():
  I = input ("Enter the Name" "\n")
  P = "[a-zA-Z]{3,20}$"
  if re.match (P,I):
    print ("Name Validated")
    return (I)
  else:
    print ("Enter the Valid Name \nLength of the Name must be 3-20")

def Roll_num():
  I = input ("Enter the Roll Number" "\n")
  P = "^[A-Z]{6}[0-9]{4}$"
  if re.match (P,I):
    print ("Roll Number Validated")
    return (I)
  else:
    print ("Enter the Valid Roll Number \nEnter the 10 Digit Roll number similar to [GUVIDS0000]")

def Dept():
  I = input ("Enter the Department Name" "\n")
  P = "^[A-Za-z]{2,10}$"
  if re.match (P,I):
    print ("Department Validated")
    return (I)
  else:
    print ("Enter the Valid Department Name \nEnter the Department Name similar to [EEE/CSE/ECE/IT/etc]")

def Mail_ID():
  I = input ("Enter the Mail ID" "\n")
  P = "^[a-z A-Z]+[\._]?[a-z A-Z 0-9 \._-]+[@]\w+[A-Z a-z]+[.][A-Z a-z]{2,3}$"
  if re.match (P,I):
    print ("Mail ID Validated")
    return (I)
  else:
    print ("Enter the Valid Mail ID \n")

def Register():
  R = {
    "_id": Phone_Num(),
    "Password":PW(),
    "Name": Student_Name(),
    "Roll_Num":Roll_num(),
    "Department":Dept(),
    "Mail": Mail_ID()
    }
  Reg.insert_one(R)
  print("Registered Successfully") 

In [ ]:
def Search_Books():
  I = input("Enter the Book Title or Book Author Name \n")
  M = {"$or":[{"Book Title":I},{"Book Author":I}]}
  X = Inv.find_one(M, {"_id" : 0, "Book Title" : 1, "Book Author" : 1})
  x = pd.DataFrame(X)
  print (x)
  if (X==None):
    print("Data not found")

def Available_Books():
  I = Inv.find({},{}).sort("_id")
  I = pd.DataFrame(I)
  print(I) 


In [ ]:
def ISBN():
  ISBN= input("Enter the 5 digits ISBN" "\n")
  M = "[0-9]{5}$"
  if re.match (M,ISBN):
    if Inv.find_one({"_id" : {"$eq" : ISBN}}):
      print("ISBN already exists")
    else:
      return (ISBN)
  else: 
    print ("Enter the Valid 5 digits ISBN")
      

In [ ]:
def Add_Books():
  n = int (input ("Enter the number of books to be added" "\n"))
  for i in range (0,n):
    T = input("Enter the Book Title" "\n")
    A = input("Enter the Book Autor" "\n")
    I = {"_id" : ISBN() , "Book Title" : T, "Book Author" : A}
    Inv.insert_one(I)
  print("Book has been added Sucessfully")


In [ ]:
def Edit_Details ():
    ISBN= input("Enter the 5 digits ISBN" "\n")
    M = "[0-9]{5}$"
    if re.match (M,ISBN):
      if Inv.find_one({"_id" : {"$eq" : ISBN}}):
        I = (input ("1 - To edit Book Title \n2 - To edit Book Author \n"))
        if I=="1":
          Inp = {"_id" : ISBN}
          CT = input("Enter the New Title")
          NT ={"$set":{"Book Title":CT}}
          Inv.update_one(Inp,NT)
          print ("Updated Successfully")
        elif I=="2":
          Inp = {"_id" : ISBN}
          CT = input("Enter the New Author" "\n")
          NT ={"$set":{"Book Author":CT}}
          Inv.update_one(Inp,NT)
          print ("Updated Successfully")
        else:
          print ("Enter the valid option")
          Edit_Details()
      else:
        print ("ISBN not found")
    else: 
      print ("Enter the Valid 5 digits ISBN")

In [ ]:
def Remove_Books():
   ISBN = input("Enter the 5 digits ISBN" "\n")
   M = "[0-9]{5}$"
   if re.match (M,ISBN):
     if Inv.find_one({"_id":{"$eq":ISBN}}):
       I ={"_id":ISBN}
       Inv.delete_one(I)
       print("Deleted successfully \n")
     else:
       print("ISBN not exists" "\n")
   else:
     print("Invalid ISBN" "\n")

In [ ]:
def Issue_Books():
  I = input("\nEnter the Registered Phone Number: " "\n")
  X = Reg.find_one({'_id':I})
  if X!= None:
    Count = Iss.count_documents({'Username':I})
    if(Count<5):
      ISBNNum = input("\nEnter the ISBN Number: " "\n")
      F = Inv.find_one({"_id":ISBNNum})
      if F != None:
        D1=(input('Enter the Date of Issue: Format : YYYY-MM-DD\n'))
        D2="^\d{4}\-(0[1-9]|1[012])\-(0[1-9]|[12][0-9]|3[01])$"
        if re.match(D2,D1):
          date = dt.datetime.strptime((D1),'%Y-%m-%d')
          data ={
                        'Username' : I,
                        'Book Title' : F["Book Title"],
                        'Book Author' : F["Book Author"],
                        'ISBN' : ISBNNum,
                        'Issue_date' : date,
                        'Date_to_Return' : date + dt.timedelta(days = 15),
                        'Penalty' : 0
               }
          h=Iss.insert_one(data)
          i=Inv.delete_one({"_id":ISBNNum})
          if h != None and i != None:
            print("Book has been Issued Successfully")
          else:
            print("The Book has not been Issued \n")
        else:
          print("Enter the Valid Format - YYYY-MM-DD\n")  
          Issue_Books()  
      else:
        print("\nTry any other books")
        Issue_Books()
    else:
      print("Maximum Limit is 5\n")
      Home()    
  else:
    Issue_Books()


In [ ]:
def View_Issued():
  I = Iss.find({},{"_id":0})
  df = pd.DataFrame(I)
  print(df)

In [ ]:
def Receive_Books():
  I = input("Enter the Registered Phone Number: " "\n")
  a = Reg.find_one({'_id':I})
  if a != None:
    ISBNNum=input("\nEnter the book ISBN number: ")
    X=Iss.find_one({"ISBN":ISBNNum,"Username":I})
    if X != None:
      D1=(input('Enter the Date of received: Format : YYYY-MM-DD\n'))
      D2="^\d{4}\-(0[1-9]|1[012])\-(0[1-9]|[12][0-9]|3[01])$"
      if re.match(D2,D1):
        date = dt.datetime.strptime((D1),'%Y-%m-%d')
        Diff = date - X['Date_to_Return']
        if Diff.days <= 0:
          print("\nNo penality")
          data={
            "_id":X["ISBN"],
            "Book Title":X["Book Title"],
            "Book Author":X["Book Author"]
            }
          e=Inv.insert_one(data)
          d=Iss.delete_one({"_id":X["_id"]})
          if (e!=None and d!=None):
            print("\nBook has been Received Successfully")
          else:
            print("Try again\n") 
            Home()          
        else:
          print("Find below the Penalty\n")
          Penalty_Collector(I,ISBNNum) 
      else:
        print("Enter the Valid Format - YYYY-MM-DD\n ")   
        Receive_Books()    
    else:
      print("Username or ISBN not found \n")
      Receive_Books()
  else:
    print("Enter the Valid Phone Number\n")
    Receive_Books()

In [ ]:
def Penalty_Collector(UN,ISBN_Num):
  A = Iss.find_one({"ISBN":ISBN_Num,"Username":UN})
  if A!=None:
     R=A["Penalty"]
     print(R,"Rs Penalty\n")
     Amount = int(input("Please Enter the Amount - \n"))
     d = Amount-R
     if(d>=0):
       print(d,"Rs \n")
       Data={
           "_id":A["ISBN"],
           "Book Title":A["Book Title"],
           "Book Author":A["Book Author"]
           }
       X=Inv.insert_one(Data)  
       Del=Iss.delete_one({"_id":A["_id"]}) 
       if(X!=None and Del!=None):
         print("\nPenalty paid and Book has been Received successfully - Thanks!!!\n")
       else:
         print("Try Again \n") 
         Home()
     else:
       d1=R-Amount
       print(d1,"Rs to be paid \n")      
  else:
    print("Enter the Valid Details\n") 

In [31]:
def Admin():
  LL = input("Enter the Librarian Login ID: " "\n")
  LP = input("Enter the Librarian Password:" "\n")
  if ((LL=="admin") and (LP == "admin")):
    print("You have been Successfully Logged in")
    Option =  (input("1 - To view the Available Books \n2 - To Add the Books \n3 - To Edit the Details \n4 - To Remove the books\n5 - To Issue the books \n6 - To View the Issued Books\n7 - To Receive the books\n8 - To Collect the Penalty \n"))
    if (Option == "1"):
      Available_Books()
    elif (Option == "2"):
      Add_Books()
    elif (Option == "3"):
      Edit_Details()
    elif (Option == "4"):
      Remove_Books()
    elif (Option == "5"):
      Issue_Books()
    elif (Option == "6"):
      View_Issued()
    elif (Option == "7"):
      Receive_Books()
    elif (Option == "8"):
      a=input("\nEnter the student phone number: ")
      b=input("\nEnter the book ISBN number: ")
      Penalty_Collector(a,b)
    else:
      print("Enter the Valid Option")
      Admin()
  else:
    print ("Enter the Valid Credentials")
    Admin()


In [ ]:
def Student_Login():
  SL = input ("Enter the Student Login" "\n")
  SP = input ("Enter the Student Password" "\n")
  if Reg.find_one({"$and":[{"_id" : {"$eq" :SL}},{"Password" : {"$eq" : SP}}]}):
    print (f"{SL} - You have been Successfully Logged in")
    Option = (input ("1 - To View the Details \n2 - To View the Available Books \n3 - To Search for the Books \n"))
    if Option == "1":
      V = Iss.find({},{"_id":0, "Book Title":1 , "Issued date":1,"Date of Return":1, "Penalty":1})
      df = pd.DataFrame(V)
      print(df)
    elif Option =="2":
      Available_Books()
    elif Option == "3":
      Search_Books()
    else:
      print ("Enter the Valid Input")
      Home()
  else:
    print("Enter the Valid Credentials")
    I = (input("1 - To Register \n2 - To login \n"))
    if(I== "1"):
      Register()
    elif(I== "2") :
      Student_Login()
    else:
      print("Enter the Valid Input")  
      Home()

In [23]:
def Penalty():
  P = Iss.find({})
  for i in P:
    D = dt.datetime.now() - i['Date_to_Return']
    if (D.days>0):
      Pe = D.days*20
      S ={"$set":{"Penalty": Pe}}
      Iss.update_one({"_id":i["_id"]},S)

def Home():
  Penalty()
  Option = (input("1 - Student Login \n2 - Librarian Login \n"))
  if Option == "1":
    print ("Welcome")
    Op = (input("1 - Student Register \n2 - Student Login \n"))
    if Op == "1":
      Register()
    elif Op == "2":
      Student_Login()
    else: 
      Home()
  elif Option == "2":
    Admin()
  else:
    Home()

In [34]:
Home()

1 - Student Login 
2 - Librarian Login 
2
Enter the Librarian Login ID: 
admin
Enter the Librarian Password:
admin
You have been Successfully Logged in
1 - To view the Available Books 
2 - To Add the Books 
3 - To Edit the Details 
4 - To Remove the books
5 - To Issue the books 
6 - To View the Issued Books
7 - To Receive the books
8 - To Collect the Penalty 
6
     Username     Book Title Book Author   ISBN Issue_date Date_to_Return  \
0  8908908900  Wings of Fire         APJ  00001 2022-10-10     2022-10-25   

   Penalty  
0      100  
